# Helmet Classification For TinyML Project

> 이 notebook 은 open source 컨트리뷰톤 2020 - tinyML (Tensorflow Lite Project) Mobility Team 의 오픈소스 프로젝트를 위해 만들어졌습니다. 

- 모빌리티 팀 (멘토 맹윤호)
- 최예진(팀장), 이민우, 전수민, 이장후, 이경환, 조승현
- **.ipynb 제작 - 이장후. 2020/08/25**
- **.ipynb 수정자 -**

<br>

- Target Github Repository : [TinyML : Tensorflow lite for microcontroller](https://github.com/yunho0130/tensorflow-lite)
- Team Github Repository : [TinyML-Mobility](https://github.com/orgs/tinyml-mobility/teams)

<br>

## Before We Start
- 런타임 -> GPU 로 변경 하셨나요?

# Google Drive

- 학습을 시키기 전 데이터가 있는 google Drive 와 연동을 해야 합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Include Library

- 이 노트북의 소스코드는 tensorflow 2.0 이상과 호환되지 않습니다.
- Google colab 에서는 %tensorflow_version 을 통해, 원하는 버전의 tensorflow 를 쉽게 불러올 수 있습니다

In [ ]:
try:
  # This %tensorflow_version magic only works in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

# For your non-Colab code, be sure you have tensorflow==1.15
import tensorflow as tf
assert tf.__version__.startswith('1')

# tf.enable_eager_execution() 테스트 안 해봄.

import os
import numpy as np
import matplotlib.pyplot as plt

TensorFlow 1.x selected.


In [ ]:
tf.__version__

'1.15.2'

## Serve Data
- 헬멧 미착용 데이터셋 : https://drive.google.com/file/d/1WYZV_UTPvjv16Z57uMm_kBSWAYJoI3jk/view?usp=sharing
- 헬멧 착용 데이터셋 : https://drive.google.com/file/d/1SrN8LPwvlTttmxJQMDBgDNs_nYpqMOxw/view?usp=sharing
- 위 데이터들을 모두 자신의 구글 드라이브에 올려 둡시다.
- 구글 드라이브에게 '바탕화면' 같은 존재는 ```/content/gdrive/"My Drive"/``` 입니다.
- 저의 경우 위 압축 파일들을 ```/content/gdrive/"My Drive"/data/``` 에 넣어 두겠습니다.

<br>

### Zip 압축 풀기
- linux 에서 zip 파일의 압축을 풀기 위해서는, unzip 명령을 사용합니다. Google Colab 에서는, ! 를 이용해서 리눅스 명령어를 호출할 수 있습니다.
- 이때, 리눅스 명령이 수행되는 위치는, 현재 위치입니다. 현재 위치는 !ls 를 통해 조회할 수 있습니다.
- %cd 를 통해, 현재 위치를 이동할 수 있습니다.

<br>

현재 위치를 조회합니다. <br>
```!ls```

<br>

압축파일이 있는 해당 위치로 이동합니다. <br>
```%cd /content/gdrive/"My Drive"/data/```

<br>

압축을 해제합니다. <br>
```!unzip <파일명>.zip -d <압축을 풀 폴더>```

<br>

약간의 시간이 소요됩니다. <br>

<br>

여담 <br>
> 프로젝트를 진행하다가, ls 와 cd 모두 linux 명령인데 왜 cd 는 % 를 사용하고 ls 는 ! 를 사용하는지 궁금할 수 있습니다. 정확히 말하면, % 는 google colab 에서 특정한 목적을 가지고 제공되는 기능으로, magic 이라고 합니다. 반면 ! 는 순수 linux 명령어입니다. !cd 를 이용해서 경로를 이동할 수 있으나, 그 경로가 유지되지 않고 원래대로 돌아와 버립니다. 따라서 cd 를 이용해서 경로 작업을 할 때에는 % 를 사용해야 합니다.

In [ ]:
!ls
%cd /content/gdrive/"My Drive"/data/

# 혹시 이전에 스크립트릴 실행했을 수 있으니, 다시 실행시키려면 그 전에 미리 잘 지워줍시다.
!rm -r helmetclassification 

# 경로들을 만들어 줍시다.
!mkdir helmetclassification
!mkdir helmetclassification/helmet
!mkdir helmetclassification/non_helmet

# 압축 해제!
!unzip helmet_crop.zip -d /content/gdrive/"My Drive"/data/helmetclassification/helmet
!unzip not_wearing_a_helmet.zip -d /content/gdrive/"My Drive"/data/helmetclassification/non_helmet

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/gdrive/My Drive/data/helmetclassification/non_helmet/__MACOSX/face_1500-5000/._003578.jpg  
  inflating: /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1500-5000/002672.jpg  
  inflating: /content/gdrive/My Drive/data/helmetclassification/non_helmet/__MACOSX/face_1500-5000/._002672.jpg  
  inflating: /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1500-5000/004203.jpg  
  inflating: /content/gdrive/My Drive/data/helmetclassification/non_helmet/__MACOSX/face_1500-5000/._004203.jpg  
  inflating: /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1500-5000/004565.jpg  
  inflating: /content/gdrive/My Drive/data/helmetclassification/non_helmet/__MACOSX/face_1500-5000/._004565.jpg  
  inflating: /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1500-5000/002114.jpg  
  inflating: /content/gdrive/My Drive/data/helmetclassification/non_helmet/__MACOSX/face_1500-50

## Hyper Parameters
- 하이퍼파라미터들을 위에 모아서 정의해두는 습관
- google colab 에서는 아까 언급했던 % magic 을 이용해서 tensorboard 를 쉽게 사용할 수 있도록 만들어 두었습니다. (그런데 잘 돌아가지는 않음)

In [ ]:
%load_ext tensorboard

In [ ]:
%cd /content/gdrive/"My Drive"/data/

# 데이터가 존재하는 경로 ( /content/gdrive/"My Drive"/data/helmetclassification ) 를 data_dir 변수에 저장합니다.
data_dir = os.path.join(os.getcwd(), 'helmetclassification')
print(data_dir)

# input 이미지의 크기는 128 by 128 by 3 으로 상정합니다. 채널은 RGB 이므로, 3 입니다.
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNEL = 3
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)

# 연산 처리 단위 (배치) 는 이미지 16장, 그리고 Learning Rate, optimizer, 에폭 등을 설정합니다.
# *참고* : Optimizer 에서 SGD 는 최근 잘 사용하지 않지만, 안정적인 수렴을 위해 특정 경우에 사용합니다. 
BATCH_SIZE = 16
LEARNING_RATE_SGD = 0.001
LEARNING_RATE_ADAM = 0.0001
TRAINING_OPTIMIZER_SGD  = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE_SGD, momentum=0.0)
TRAINING_OPTIMIZER_ADAM = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_ADAM)
EPOCHS = 3

# 우리는 h5 file 을 tflite 파일로 전환할 것입니다. 이름을 미리 정해 둡시다.
SAVED_KERAS_MODEL_NAME = 'helmet_classification_model.h5'

# 우리는 잠시 후에 라벨 파일을 만들어낼 것인데, 라벨 파일의 이름을 미리 정의해 둡시다.
LABEL_FILE_NAME = 'ishelmetlabel.txt'

/content/gdrive/My Drive/data
/content/gdrive/My Drive/data/helmetclassification


## Generator
- python 에는 '제네레이터' 라는 것이 존재합니다.
- 그 때 그 때, 상황에 맞는 데이터를 만들어 주는 역할을 수행합니다.
- keras 에서는, 이를 이용하여, 우리가 가지고 있는 image data 들을 정갈하게 정리해서 하나씩 하나씩 필요에 맞게 넘겨 주는 함수를 만들어 두었습니다. 

<br>

### ImageDataGenerator : Data Augmentation
- Data Augmentation 은 말 그대로 데이터를 증폭시켜주는 역할을 합니다.
- 고양이 사진이 있을 때, 고양이 머리만 있어도 고양이이고, 몸통만 있어도 고양이이며, 형광등 아래의 고양이도 고양이고, 백열등 아래의 고양이도 고양이입니다.
- 우리가 가지고 있는 이미지를 인위적으로 변형한 뒤에도, 올바르게 추론할 수 있도록 이미지 데이터를 다양한 방법으로 변형시켜줄 수 있도록 돕는 함수가 Keras 의 ImageDataGenerator 입니다.
- Train 할 때는 이미지를 변형해서 forward-pass 시켜주지만, test 를 할 때에는 이미지를 변형해서 네트워크를 통과시키면 안 되겠지요.
- 다만, Train 이나 Test 시에는 Network 의 사양에 맞추어서 0~1 의 범위로 input 을 정규화 시켜 주어야 합니다.

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rescale=1./255,
    dtype='float',
    fill_mode="nearest",
    horizontal_flip=True,
    channel_shift_range=0.1, 
    rotation_range=20,
    brightness_range=(-0.3, 0.3)
    )

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    
    dtype='float',
    )

## Train-Valid(Test-Validation) Data Split

- keras 는 image classification 과 같은 간단한 문제들을 쉽게 해결할 수 있도록 돕기 위해 ```flow_from_directory``` 와 같은 함수를 제공합니다.
- 위에서 만들었던 ImageDataGenerator() 을 거친 객체들을 ```flow_from_directory``` 에 대입하여 간단하게 클래스별로 학습을 시킬 수 있습니다.
- 이때 Keras 는 미리 데이터를 '/train' 폴더와 '/test' 폴더로 나누어 놓기를 요구합니다.
- 하지만 우리는 train data 와 test data 를 나누어두지 않았습니다. 이 경우에도, Keras 는 우리의 작업을 도와줄 수 있는 함수를 제공하지만, 해당 함수를 사용하면, Data Augmentation 을 정상적으로 수행할 수 없습니다.
- 따라서, 우리가 직접 모은 데이터들을 train 과 test 폴더로 분리해서 담아 주는 스크립트를 작성해 봅시다.

*Data Augmentation 과 train_test split 사이의 문제 : 궁금하신 분들은 참고 : [관련 이슈](https://stackoverflow.com/questions/53037510/can-flow-from-directory-get-train-and-validation-data-from-the-same-directory-in)*

In [ ]:
# 현재 디렉터리 이동
%cd /content/gdrive/"My Drive"/data/helmetclassification/

# /content/gdrive/"My Drive"/data/helmetclassification/ 에 train 과 test 폴더를 만들어 줍니다.
!mkdir train
!mkdir train/helmet
!mkdir train/non_helmet
!mkdir test
!mkdir test/helmet
!mkdir test/non_helmet

/content/gdrive/My Drive/data/helmetclassification


### Wearing Helmet Data
- test 데이터 랜덤하게 뽑아내기
- 뽑은 데이터 test 폴더로 옮겨주기
- 나머지 데이터를 train 폴더로 옮기기

In [ ]:
# /content/gdrive/"My Drive"/data/helmetclassification/helmet 경로부터 작업해 줍시다.
%cd /content/gdrive/"My Drive"/data/helmetclassification/helmet

helmet_data_directory = os.getcwd()
file_list = os.listdir(helmet_data_directory)
print('Count of file :', len(file_list))

import random
sampled_helmet_file_list = random.sample(file_list, int(len(file_list) * 0.4))
print('Count of sampled file :', len(sampled_helmet_file_list))

/content/gdrive/My Drive/data/helmetclassification/helmet
Count of file : 12772
Count of sampled file : 5108


In [ ]:
# 이제 sample 된 데이터를 /content/gdrive/"My Drive"/data/helmetclassification/test/helmet 으로 옮겨 줄 차례입니다.
import shutil
target_dir = os.path.join(os.path.join(data_dir, 'test'), 'helmet')
for f in sampled_helmet_file_list :
  current_file = os.path.join(helmet_data_directory, f)
  shutil.move(current_file, os.path.join(target_dir, f))
  print(current_file,'\t->\t',os.path.join(target_dir, f))

# 이제 test 데이터를 모두 옮겼으니, train 데이터도 모두 옮겨 줍시다.
%cd /content/gdrive/"My Drive"/data/helmetclassification/
!mv helmet/ train/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/gdrive/My Drive/data/helmetclassification/helmet/774_0.png 	->	 /content/gdrive/My Drive/data/helmetclassification/test/helmet/774_0.png
/content/gdrive/My Drive/data/helmetclassification/helmet/2625_2.png 	->	 /content/gdrive/My Drive/data/helmetclassification/test/helmet/2625_2.png
/content/gdrive/My Drive/data/helmetclassification/helmet/2025_6.png 	->	 /content/gdrive/My Drive/data/helmetclassification/test/helmet/2025_6.png
/content/gdrive/My Drive/data/helmetclassification/helmet/1667_19.png 	->	 /content/gdrive/My Drive/data/helmetclassification/test/helmet/1667_19.png
/content/gdrive/My Drive/data/helmetclassification/helmet/910_1.png 	->	 /content/gdrive/My Drive/data/helmetclassification/test/helmet/910_1.png
/content/gdrive/My Drive/data/helmetclassification/helmet/3138_1.png 	->	 /content/gdrive/My Drive/data/helmetclassification/test/helmet/3138_1.png
/content/gdrive/My Drive/data/helmetclassification/helmet/2694_0.png 	->	 /con

### Not Wearing Helmet Data
- test 데이터 랜덤하게 뽑아내기
- 뽑은 데이터 test 폴더로 옮겨주기
- 나머지 데이터를 train 폴더로 옮기기

In [ ]:
# /content/gdrive/"My Drive"/data/helmetclassification/non_helmet 경로를 작업할 차례입니다.
%cd /content/gdrive/"My Drive"/data/helmetclassification/non_helmet

# 해당 경로는, 다양한 경로로부터 사람의 이미지를 취득했기에, 이를 모두 모아주는 작업이 필요합니다.
file_list = os.listdir(os.getcwd())
print(file_list)

# 내부에 MACOS 용 파일과 자잘한 더미 파일이 존재하니 이를 제거해 줍니다.
!rm -r __MACOSX
!rm ./pedestrian/.DS_Store
!ls

/content/gdrive/My Drive/data/helmetclassification/non_helmet
['face_1-1500', '__MACOSX', 'face_1500-5000', 'pedestrian']
face_1-1500  face_1500-5000  pedestrian


In [ ]:
%cd /content/gdrive/"My Drive"/data/helmetclassification/non_helmet

file_list = []
for (root, dirs, files) in os.walk(os.getcwd()):
  print("root : " + root)
  if len(files) > 0:
    for file_name in files:
      if file_name[-3:] in ['jpg', 'png'] :
        file_full_path = os.path.join(root, file_name)
        print('append to file_list : ', file_full_path)
        file_list.append(file_full_path)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
append to file_list :  /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000932.jpg
append to file_list :  /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000926.jpg
append to file_list :  /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000098.jpg
append to file_list :  /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/001386.jpg
append to file_list :  /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000927.jpg
append to file_list :  /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/001387.jpg
append to file_list :  /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000099.jpg
append to file_list :  /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/001393.jpg
append to file_list :  /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000

In [ ]:
import random
sampled_nonhelmet_file_list = random.sample(file_list, int(len(file_list) * 0.4))
print('Count of sampled file :', len(sampled_nonhelmet_file_list), 'out of', len(file_list))

Count of sampled file : 2360 out of 5901


In [ ]:
# 이제 sample 된 데이터를 /content/gdrive/"My Drive"/data/helmetclassification/test/helmet 으로 옮겨 줄 차례입니다.
import shutil
target_dir = os.path.join(os.path.join(data_dir, 'test'), 'non_helmet')
for cnt, f in enumerate(sampled_nonhelmet_file_list) :
  # 새로운 임시 이름 (cnt) + 확장자 (.xxx) 로 새로운 이름을 부여
  postfix = str(cnt) + f[-4:] 
  shutil.move(f, os.path.join(target_dir, postfix))
  print('[test data]', f,'\t->\t',os.path.join(target_dir, postfix))

[test data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000750.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/test/non_helmet/0.jpg
[test data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1500-5000/001529.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/test/non_helmet/1.jpg
[test data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000785.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/test/non_helmet/2.jpg
[test data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000409.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/test/non_helmet/3.jpg
[test data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/001198.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/test/non_helmet/4.jpg
[test data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1500-5000/003434.jpg 	->	 /content/gdrive/M

In [ ]:
# test 데이터를 모두 옮겼으니, train 데이터도 모두 옮겨 줍시다.
cnt = 0
target_dir = os.path.join(os.path.join(data_dir, 'train'), 'non_helmet')
for f in file_list:
  if os.path.isfile(f):
    postfix = str(cnt) + f[-4:]
    shutil.move(f, os.path.join(target_dir, postfix))
    print('[train data]', f,'\t->\t',os.path.join(target_dir, postfix))
    cnt += 1

# 원래 존재하던 폴더를 지워 버립시다.
# %cd /content/gdrive/"My Drive"/data/helmetclassification/
# !rm -r non_helmet

[train data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/001264.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/train/non_helmet/0.jpg
[train data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000634.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/train/non_helmet/1.jpg
[train data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000152.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/train/non_helmet/2.jpg
[train data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000813.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/train/non_helmet/3.jpg
[train data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/000185.jpg 	->	 /content/gdrive/My Drive/data/helmetclassification/train/non_helmet/4.jpg
[train data] /content/gdrive/My Drive/data/helmetclassification/non_helmet/face_1-1500/001072.jpg 	->	 /content/gdr

### Check Directory
- 잘 정리되었나 확인해 봅시다
- train, test 폴더만 남아 있어야 합니다.

In [ ]:
%cd /content/gdrive/"My Drive"/data/helmetclassification/
!ls

# helmet
%cd /content/gdrive/"My Drive"/data/helmetclassification/train/helmet/
file_list = os.listdir(os.getcwd())
print('count of train helmet file :', len(file_list))
%cd /content/gdrive/"My Drive"/data/helmetclassification/test/helmet/
file_list = os.listdir(os.getcwd())
print('count of test helmet file :', len(file_list))

# non_helmet
%cd /content/gdrive/"My Drive"/data/helmetclassification/train/non_helmet/
file_list = os.listdir(os.getcwd())
print('Count of train non_helmet file :', len(file_list))
%cd /content/gdrive/"My Drive"/data/helmetclassification/test/non_helmet/
file_list = os.listdir(os.getcwd())
print('Count of test non_helmet file :', len(file_list))

/content/gdrive/My Drive/data/helmetclassification
non_helmet  test  train
/content/gdrive/My Drive/data/helmetclassification/train/helmet
count of train helmet file : 7664
/content/gdrive/My Drive/data/helmetclassification/test/helmet
count of test helmet file : 5108
/content/gdrive/My Drive/data/helmetclassification/train/non_helmet
Count of train non_helmet file : 3541
/content/gdrive/My Drive/data/helmetclassification/test/non_helmet
Count of test non_helmet file : 2360


## Data Generator Flow From Directory
- Keras 는 Directory 이름과 해당 폴더로부터 알아서 클래스를 구분해주고, 쉽게 training 시킬 수 있도록 API 를 제공합니다.
- train_generator : generator 로써, next() 수행 시 하나의 tensor 를 만들어냅니다.
  - 한 번에 만들어내는 tensor 은 (image tensor, label tensor) 이고, 그 shape 은 (batch_size, 224, 224, 3)  (batch_size, 2 - 클래스가 2개이므로) 입니다.
- val_generator : generator 로써, 이하동문.
  - 한 번에 만들어내는 tensor 은 (image tensor, label tensor) 이고, 그 shape 은 (1, 224, 224, 3)  (1, 2) 입니다.


In [ ]:
train_generator = train_datagen.flow_from_directory(
    data_dir+'/train',
    color_mode="rgb",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    )

val_generator = test_datagen.flow_from_directory(
    data_dir+'/test',
    color_mode="rgb",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=1,
    )

Found 11205 images belonging to 2 classes.
Found 7468 images belonging to 2 classes.


### Label 생성기
- 우리가 데이터들을 (non_helmet, helmet) 클래스별로 train, test 로 나누고 깔끔하게 데이터를 전부 모아 두었으므로, 해당 폴더 이름으로부터 label 파일을 만들 수 있습니다.
- label 파일이란, 모델이 추론한 결과가 어떤 클래스를 의미하는지 정보를 담고 있는 그냥 이름표 파일이라고 생각하면 됩니다.


In [ ]:
# 디렉토리 이름을 라벨로 바로 써버리는 소스코드입니다.
%cd /content/gdrive/"My Drive"/data/

print(train_generator.class_indices) # type dictionary
print(len(train_generator.class_indices))
class_cnt = len(train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('{}'.format(LABEL_FILE_NAME), 'w') as f:
  f.write(labels)

!cat {LABEL_FILE_NAME}

/content/gdrive/My Drive/data
{'helmet': 0, 'non_helmet': 1}
2
helmet
non_helmet

# Model For training
- 이제 모델을 만들고, 훈련을 시켜 보도록 합시다.

In [ ]:
# Create the base model from the pre-trained MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
base_model.trainable = True

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
9412608/9406464 [==============================] - 1s 0us/step


In [ ]:
# 뒷단에 추가 레이어
full_model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(name = "GAP"),
  # tf.keras.layers.Convolution2D(2, (1,1), strides=(1,1), activation='softmax', name='final_dense'),
  tf.keras.layers.Dense(units=2, activation='softmax', name = "final_dense"),
])

In [ ]:
full_model.compile(optimizer=TRAINING_OPTIMIZER_SGD, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
full_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Model) (None, 4, 4, 1280)        2257984   
_________________________________________________________________
GAP (GlobalAveragePooling2D) (None, 1280)              0         
_________________________________________________________________
final_dense (Dense)          (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,226,434
Non-trainable params: 34,112
_________________________________________________________________


In [ ]:
dense_layer = full_model.layers[-1].get_weights()[0]
gap_layer = full_model.layers[-2].output
final_conv_layer = full_model.layers[-3].output
print(type(dense_layer), type(gap_layer), type(final_conv_layer))
print('dense layer W shape : ', dense_layer.shape)
print('GAP layer output tensor shape : ', gap_layer.shape)
print('final convolution layer output tensor shape : ', final_conv_layer.shape)

<class 'numpy.ndarray'> <class 'tensorflow.python.framework.ops.Tensor'> <class 'tensorflow.python.framework.ops.Tensor'>
dense layer W shape :  (1280, 2)
GAP layer output tensor shape :  (?, 1280)
final convolution layer output tensor shape :  (?, 4, 4, 1280)


## Train
- 모델을 다 만들었으니 이제 훈련을 시킬 차례입니다.
- 그런데 훈련을 시작하게 되면, 개입이 불가능해집니다.
- 따라서 특정 타이밍(예를 들어, 1에폭마다) 적절히 호출되는 콜백 함수를 미리 등록시켜 두고, 훈련을 시작하는 것이 바람직합니다.
  - 중간중간 모델의 가중치를 세이브하는 기능
  - 텐서보드로 로그를 전송하는 기능
- 일반적으로 이 두 가지 기능을 callback 으로 등록시켜 두곤 합니다.
- keras_model.fit_generator() 함수가 호출되면 Training 이 시작됩니다.


In [ ]:
testdata = val_generator.next()[2]
print(testdata.shape, testdata.dtype)
traindata = train_generator.next()[2]
print(traindata.shape, testdata.dtype)

IndexError: ignored

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        data = train_generator.next()
        print(len(data), type(data))
        shape1 = data[0].shape
        answer_shape1 = data[1].shape
        dtype1 = data[0].dtype
        print("...Training: start of batch {}; shape {} - dtype {}, {}; got log keys: {}\n".format(batch, shape1, dtype1, answer_shape1, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        data = val_generator.next()
        shape1 = data[0].shape
        answer_shape1 = data[1].shape
        dtype1 = data[0].dtype
        print("...Evaluating: start of batch {}; shape {} - dtype {}, {}; got log keys: {}\n".format(batch, shape1, dtype1, answer_shape1, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))

In [ ]:
import datetime
%cd /content/gdrive/"My Drive"/data/helmetclassification

# 저장할 로그 파일의 이름을 준비합니다.
log_dir = os.path.join(os.getcwd(), os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,     # 저장할 로그파일의 이름을 지정합니다.
                                                      histogram_freq=1,    # 가중치나 이런 것들을 visualization 하는 histogram 을 그리는 작업을 1 epoch 마다 합니다. 
                                                      update_freq='epoch'
                                                      ) # 1에폭마다 Tensorboard 를 갱신합니다.

# 학습 시작. 변수 history 는 학습을 마친 뒤 학습중 있었던 일들을 볼 수 있도록 keras 에서 제공하는 객체입니다.
history = full_model.fit_generator(train_generator, 
                    epochs=EPOCHS, 
                    validation_data=val_generator,
                    callbacks=[
                               #tensorboard_callback, 
                               CustomCallback()],
                    )

/content/gdrive/My Drive/data/helmetclassification
Start epoch 0 of training; got log keys: []
Epoch 1/3
2 <class 'tuple'>
...Training: start of batch 0; shape (16, 128, 128, 3) - dtype float64, (16, 2); got log keys: ['batch', 'size']

  1/701 [..............................] - ETA: 5:26 - loss: 0.3377 - acc: 0.87502 <class 'tuple'>
...Training: start of batch 1; shape (16, 128, 128, 3) - dtype float64, (16, 2); got log keys: ['batch', 'size']

  2/701 [..............................] - ETA: 4:19 - loss: 0.4412 - acc: 0.78122 <class 'tuple'>
...Training: start of batch 2; shape (16, 128, 128, 3) - dtype float64, (16, 2); got log keys: ['batch', 'size']

  3/701 [..............................] - ETA: 3:56 - loss: 0.4050 - acc: 0.79172 <class 'tuple'>
...Training: start of batch 3; shape (16, 128, 128, 3) - dtype float64, (16, 2); got log keys: ['batch', 'size']

  4/701 [..............................] - ETA: 3:48 - loss: 0.3719 - acc: 0.82812 <class 'tuple'>
...Training: start of bat

InvalidArgumentError: ignored

## Estimate
- train 의 과정과 결과를 보는 방법은, tensorboard 를 활용하는 방법도 있고, history 객체를 이용하는 방법도 있습니다.

In [ ]:
# history 객체 이용
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(19, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

KeyError: ignored

In [ ]:
%tensorboard --logdir logs

### Estimate with Testset
- 학습 과정에서의 변화를 관찰했다면, 이제 우리가 아직 사용하지 않은 test set 을 활용해서 학습 결과를 관찰해 보도록 합시다.
- 제네레이터에서 제공되는 샘플로 평가할 때는 evaluate_generator 함수를 사용합니다.

In [ ]:
scores  = tf.keras.evaluate_generator(
  full_model,
  val_generator,
  steps = 100,
  workers = 4,
  callbacks = None
)
print(scores, " % Accuracy")

# Save Entire Model
- keras 는 다시 keras 를 이용해서 쉽게 모델을 불러올 수 있도록 저장하는 API 를 가지고 있습니다.
- 저장합시다!

In [ ]:
%cd /content/gdrive/"My Drive"/data/helmetclassification
entire_model.save(SAVED_KERAS_MODEL_NAME)

# Next Time (part2)
수고했습니다! Part 2 에서는 다음과 같은 내용을 진행해 보도록 합시다.
- keras 내장 모델인 mobilenet v2 를 불러오지 말고, 직접 mobilenet v2 를 구현해 봅시다.
- 구현된 모델을 조금 수정하면서, Class Activation Map* 을 한번 visualization 해 봅시다.

*Class Activation Map 이란, Helmet 클래스로 판단하는 데 어떤 부분을 가장 주목해서 보았는지와 같이, 어떤 클래스로 판단하는 것의 근거를 Visualization 한 이미지를 의미합니다. 